In [1]:
"""
Created on Fri Feb  1 17:31:47 2019

@author: Chris
"""
#import libraries
import numpy as np
import numpy.lib.recfunctions as rfn # used to manipulate structured arrays
from collections import OrderedDict # dictionary subclass that remembers the
                                    # order that the keys were first inserted

#NumPy Cheatsheet - https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf


## HW notes:
'''
A medical claim is denoted by a claim number ('Claim.Number'). Each claim consists of one or more medical lines denoted by a claim line number ('Claim.Line.Number').

1. J-codes are procedure codes that start with the letter 'J'.

     A. Find the number of claim lines that have J-codes.

     B. How much was paid for J-codes to providers for 'in network' claims?

     C. What are the top five J-codes based on the payment to providers?



2. For the following exercises, determine the number of providers that were paid for at least one J-code. Use the J-code claims for these providers to complete the following exercises.

    A. Create a scatter plot that displays the number of unpaid claims (lines where the ‘Provider.Payment.Amount’ field is equal to zero) for each provider versus the number of paid claims.

    B. What insights can you suggest from the graph?

    C. Based on the graph, is the behavior of any of the providers concerning? Explain.



3. Consider all claim lines with a J-code.

     A. What percentage of J-code claim lines were unpaid?

     B. Create a model to predict when a J-code is unpaid. Explain why you choose the modeling approach.

     C. How accurate is your model at predicting unpaid claims?

      D. What data attributes are predominately influencing the rate of non-payment?
'''

"\nA medical claim is denoted by a claim number ('Claim.Number'). Each claim consists of one or more medical lines denoted by a claim line number ('Claim.Line.Number').\n\n1. J-codes are procedure codes that start with the letter 'J'.\n\n     A. Find the number of claim lines that have J-codes.\n\n     B. How much was paid for J-codes to providers for 'in network' claims?\n\n     C. What are the top five J-codes based on the payment to providers?\n\n\n\n2. For the following exercises, determine the number of providers that were paid for at least one J-code. Use the J-code claims for these providers to complete the following exercises.\n\n    A. Create a scatter plot that displays the number of unpaid claims (lines where the ‘Provider.Payment.Amount’ field is equal to zero) for each provider versus the number of paid claims.\n\n    B. What insights can you suggest from the graph?\n\n    C. Based on the graph, is the behavior of any of the providers concerning? Explain.\n\n\n\n3. Conside

In [2]:
#Read the two first two lines of the file.
with open('claim.sample.csv', 'r') as f:
    print(f.readline())
    print(f.readline())

V1,Claim.Number,Claim.Line.Number,Member.ID,Provider.ID,Line.Of.Business.ID,Revenue.Code,Service.Code,Place.Of.Service.Code,Procedure.Code,Diagnosis.Code,Claim.Charge.Amount,Denial.Reason.Code,Price.Index,In.Out.Of.Network,Reference.Index,Pricing.Index,Capitation.Index,Subscriber.Payment.Amount,Provider.Payment.Amount,Group.Index,Subscriber.Index,Subgroup.Index,Claim.Type,Claim.Subscriber.Type,Claim.Pre.Prince.Index,Claim.Current.Status,Network.ID,Agreement.ID

1,28849991,1,9583423,FA0001411002,2,128,RR8I, , ,V5789,19356.96, ,A,I,N,N,N,0,15036.21,110,7763000,0,M,H, ,2,P00000000002,FH001310QB01



In [3]:
#Colunn names that will be used in the below function, np.genfromtxt
names = ["V1","Claim.Number","Claim.Line.Number",
         "Member.ID","Provider.ID","Line.Of.Business.ID",
         "Revenue.Code","Service.Code","Place.Of.Service.Code",
         "Procedure.Code","Diagnosis.Code","Claim.Charge.Amount",
         "Denial.Reason.Code","Price.Index","In.Out.Of.Network",
         "Reference.Index","Pricing.Index","Capitation.Index",
         "Subscriber.Payment.Amount","Provider.Payment.Amount",
         "Group.Index","Subscriber.Index","Subgroup.Index",
         "Claim.Type","Claim.Subscriber.Type","Claim.Pre.Prince.Index",
         "Claim.Current.Status","Network.ID","Agreement.ID"]

In [4]:
#https://docs.scipy.org/doc/numpy-1.12.0/reference/arrays.dtypes.html
#These are the data types or dtypes that will be used in the below function, np.genfromtxt()
types = ['S8', 'f8', 'i4', 'i4', 'S14', 'S6', 'S6', 'S6', 'S4', 'S9', 'S7', 'f8',
         'S5', 'S3', 'S3', 'S3', 'S3', 'S3', 'f8', 'f8', 'i4', 'i4', 'i4', 'S3',
         'S3', 'S3', 'S4', 'S14', 'S14']

In [5]:
#NumPy Structured Arrays: https://docs.scipy.org/doc/numpy/user/basics.rec.html
# Though... I like this Structured Array explanation better in some cases: https://jakevdp.github.io/PythonDataScienceHandbook/02.09-structured-data-numpy.html
#np.genfromtxt:  https://docs.scipy.org/doc/numpy/reference/generated/numpy.genfromtxt.html

#read in the claims data into a structured numpy array
CLAIMS = np.genfromtxt('claim.sample.csv', dtype=types, delimiter=',', names=True,
                       usecols=[0,1,2,3,4,5,
                                6,7,8,9,10,11,
                                12,13,14,15,16,
                                17,18,19,20,21,
                                22,23,24,25,26,
                                27,28])


In [6]:
#print dtypes and field names
print(CLAIMS.dtype)

[('V1', 'S8'), ('ClaimNumber', '<f8'), ('ClaimLineNumber', '<i4'), ('MemberID', '<i4'), ('ProviderID', 'S14'), ('LineOfBusinessID', 'S6'), ('RevenueCode', 'S6'), ('ServiceCode', 'S6'), ('PlaceOfServiceCode', 'S4'), ('ProcedureCode', 'S9'), ('DiagnosisCode', 'S7'), ('ClaimChargeAmount', '<f8'), ('DenialReasonCode', 'S5'), ('PriceIndex', 'S3'), ('InOutOfNetwork', 'S3'), ('ReferenceIndex', 'S3'), ('PricingIndex', 'S3'), ('CapitationIndex', 'S3'), ('SubscriberPaymentAmount', '<f8'), ('ProviderPaymentAmount', '<f8'), ('GroupIndex', '<i4'), ('SubscriberIndex', '<i4'), ('SubgroupIndex', '<i4'), ('ClaimType', 'S3'), ('ClaimSubscriberType', 'S3'), ('ClaimPrePrinceIndex', 'S3'), ('ClaimCurrentStatus', 'S4'), ('NetworkID', 'S14'), ('AgreementID', 'S14')]


In [7]:
#Notice the shape differs since we're using structured arrays.
print(CLAIMS.shape)

(472559,)


In [8]:
#However, you can still subset it to get a specific row.
print(CLAIMS[0])

(b'1', 28849991., 1, 9583423, b'FA0001411002', b'2', b'128', b'RR8I', b' ', b' ', b'V5789', 19356.96, b' ', b'A', b'I', b'N', b'N', b'N', 0., 15036.21, 110, 7763000, 0, b'M', b'H', b' ', b'2', b'P00000000002', b'FH001310QB01')


In [9]:
#Subset it to get a specific value.
print(CLAIMS[0][1])

28849991.0


In [10]:
#Get the names
print(CLAIMS.dtype.names)

('V1', 'ClaimNumber', 'ClaimLineNumber', 'MemberID', 'ProviderID', 'LineOfBusinessID', 'RevenueCode', 'ServiceCode', 'PlaceOfServiceCode', 'ProcedureCode', 'DiagnosisCode', 'ClaimChargeAmount', 'DenialReasonCode', 'PriceIndex', 'InOutOfNetwork', 'ReferenceIndex', 'PricingIndex', 'CapitationIndex', 'SubscriberPaymentAmount', 'ProviderPaymentAmount', 'GroupIndex', 'SubscriberIndex', 'SubgroupIndex', 'ClaimType', 'ClaimSubscriberType', 'ClaimPrePrinceIndex', 'ClaimCurrentStatus', 'NetworkID', 'AgreementID')


In [11]:
#Subset into a column
print(CLAIMS['MemberID'])

[ 9583423 25210465 25210465 ... 98799627 98799627 99975920]


In [12]:
#Subset into a column and a row value
print(CLAIMS[0]['MemberID'])

9583423


In [13]:
#String Operations in NumPy - https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.char.html

#Sorting, Searching, and Counting in NumPy - https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.sort.html

# You might see issues here: https://stackoverflow.com/questions/23319266/using-numpy-genfromtxt-gives-typeerror-cant-convert-bytes-object-to-str-impl

# If you do, encode as a unicode byte object
#A test string
test = 'J'
test = test.encode()

In [14]:
#A test NumPy array of type string
testStrArray = np.array(['Ja','JA', 'naJ', 'na' ],dtype='S9')

In [15]:
#Showing what the original string array looks like
print('Original String Array: ', testStrArray)

Original String Array:  [b'Ja' b'JA' b'naJ' b'na']


In [16]:
#Now try using startswith()
Test1Indexes = np.core.defchararray.startswith(testStrArray, test, start=0, end=None)
testResult1 = testStrArray[Test1Indexes]

In [17]:
#Showing what the original subset string array looks like with startswith()
print('Subset String Array with startswith(): ', testResult1)

Subset String Array with startswith():  [b'Ja' b'JA']


In [18]:
#Now try using find()
TestIndexes = np.flatnonzero(np.core.defchararray.find(testStrArray,test)!=-1)

testResult2 = testStrArray[TestIndexes]

In [19]:
#Showing what the original subset string array looks like with find()
print('Subset String Array with find(): ', testResult2)

Subset String Array with find():  [b'Ja' b'JA' b'naJ']


In [20]:
#Try startswith() on CLAIMS
#JcodeIndexes = np.flatnonzero(np.core.defchararray.startswith(CLAIMS['ProcedureCode'], test, start=0, end=1)!=-1)

np.set_printoptions(threshold=500, suppress=True)

In [21]:
#Using those indexes, subset CLAIMS to only Jcodes
#Jcodes = CLAIMS[JcodeIndexes]

In [22]:
#print(Jcodes)

[(b'1', 2.8849991e+07,  1,  9583423, b'FA0001411002', b'2', b'128', b'RR8I', b' ', b' ', b'V5789', 19356.96  , b' ', b'A', b'I', b'N', b'N', b'N', 0., 15036.21   ,   110,  7763000,     0, b'M', b'H', b' ', b'2', b'P00000000002', b'FH001310QB01')
 (b'2', 8.1404290e+07,  1, 25210465, b'FA0001411002', b'1', b'24', b'AL7I', b' ', b'A0205', b'V5789',     0.    , b' ', b'A', b'I', b'N', b'N', b'N', 0.,     0.     ,   110, 20421600,     0, b'M', b'H', b' ', b'91', b'P00000000007', b'FH002451QPD1')
 (b'3', 8.1404290e+07,  2, 25210465, b'FA0001411002', b'1', b'128', b'RR8I', b' ', b' ', b'V5789', 27099.744 , b'PAI', b'A', b'I', b'N', b'N', b'N', 0., 27525.26381,   110, 20421600,     0, b'M', b'H', b' ', b'91', b'P00000000007', b'FH002451QPD1')
 ...
 (b'79217', 8.9558799e+08, 19, 98799627, b'FA0001389001', b' ', b'260', b'IH7O', b' ', b' ', b'72930',   317.2665, b' ', b' ', b' ', b' ', b' ', b' ', 0.,     0.     , 48549, 80032100, 52255, b'M', b'H', b' ', b'2', b' ', b'')
 (b'103209', 8.9558799e

In [27]:
#Try find() on CLAIMS
JcodeIndexes = np.flatnonzero(np.core.defchararray.find(CLAIMS['ProcedureCode'], test, start=0, end=1)!=-1)

In [28]:
#Using those indexes, subset CLAIMS to only Jcodes
Jcodes = CLAIMS[JcodeIndexes]

print(Jcodes)

print(Jcodes.dtype.names)

[(b'3074', 7.4645490e+07, 10,  1992052, b'FA0001389001', b'5', b'636', b'OP7O', b' ', b'J1650', b'4359', 592.201995, b'KAO', b'A', b'I', b'N', b'N', b' ', 0., 0.,   114,  1613650,     0, b'M', b'H', b'S', b'91', b'P00000000001', b'FH001309QB01')
 (b'3092', 1.8860690e+07,  8, 22808129, b'FA0001389001', b'1', b'636', b'OP7O', b' ', b'J2405', b'64303', 118.83297 , b'J2L', b'A', b'I', b'N', b'N', b'R', 0., 0.,   275, 18475600,   483, b'M', b'H', b' ', b'2', b'H00000000001', b'FH001886QH01')
 (b'3093', 1.8860690e+07,  9, 22808129, b'FA0001389001', b'1', b'636', b'OP7O', b' ', b'J7030', b'64303', 196.890405, b'J2L', b'A', b'I', b'N', b'N', b'R', 0., 0.,   275, 18475600,   483, b'M', b'H', b' ', b'2', b'H00000000001', b'FH001886QH01')
 ...
 (b'288476', 4.0687899e+08,  5, 95251427, b'FA0001387001', b' ', b'636', b'OP7O', b' ', b'J7120', b'3669', 119.1292  , b' ', b' ', b' ', b' ', b' ', b' ', 0., 0., 48549, 77157900, 52255, b'M', b'H', b' ', b'2', b' ', b'')
 (b'288477', 4.0687899e+08,  6, 952

In [29]:
#QUESTION: How do you find the number of claim lines that have J-codes with "Jcodes"?
#You can figure this out. :)
print("Number of claim lines with J-Codes: " + str(Jcodes.shape[0]))

Number of claim lines with J-Codes: 51029


In [33]:
#QUESTION: How much was paid for J-codes to providers for 'in network' claims?
#Give this a try on your own after viewing the example code below.
Sorted_Jcodes = np.sort(Jcodes, order='ProviderPaymentAmount')
Sorted_Jcodes = Sorted_Jcodes[::-1]
print(Sorted_Jcodes[:10])
arrays = [Jcodes, ProviderPayments]
Jcodes_with_ProviderPayments = rfn.merge_arrays(arrays, flatten = True, usemask = False)
JCode_dict = {}
for aJCode in Jcodes_with_ProviderPayments:
    if aJCode[0] in JCode_dict.keys():
        JCode_dict[aJCode[0]] += aJCode[1]
    else:
        aJCode[0] not in JCode_dict.keys()
        JCode_dict[aJCode[0]] = aJCode[1]
np.sum([v1 for k1,v1 in JCode_dict.items()])
JCodes_PaymentsAgg_descending = OrderedDict(sorted(JCode_dict.items(), key=lambda aJCode: aJCode[1], reverse=True))
print(JCodes_PaymentsAgg_descending)

[(b'361700', 1.20381690e+08,  9, 81274542, b'FA1000015001', b'13', b'636', b'OP7O', b' ', b'J9310', b'7140', 30541.53   , b' ', b'A', b'I', b'N', b'N', b' ', 0., 30541.53   , 48546, 65836000, 0, b'E', b'H', b' ', b'11', b'MIMAPD000002', b'A00BLCH0Q001')
 (b'361699', 1.20381690e+08,  8, 81274542, b'FA1000015001', b'13', b'636', b'OP7O', b' ', b'J9310', b'7140', 30541.53   , b' ', b'A', b'I', b'N', b'N', b' ', 0., 30541.53   , 48546, 65836000, 0, b'E', b'H', b' ', b'11', b'MIMAPD000002', b'A00BLCH0Q001')
 (b'356744', 1.20454790e+08,  5, 81041406, b'FA1000015001', b'13', b'636', b'OP7O', b' ', b'J9310', b'7140', 29745.2775 , b' ', b'A', b'I', b'N', b'N', b' ', 0., 29745.2775 , 48546, 65647150, 0, b'E', b'H', b' ', b'11', b'MIMAPD000001', b'A00BLCH0Q001')
 (b'165695', 6.83342990e+08,  1,  1987669, b'FA0001387002', b'5', b'250', b'OP7O', b' ', b'J2001', b'V5332',   253.68975, b'K62', b'A', b'I', b'N', b'N', b' ', 0., 19960.58112,   114,  1610100, 0, b'M', b'H', b'S', b'2', b'P00000000001', 

NameError: name 'ProviderPayments' is not defined

array([b'1', b'11', b'15', b'19', b'2', b'81', b'89', b'91', b'99'],
      dtype='|S4')